In [ ]:
%load_ext autoreload

%autoreload 2
import os
cwd = os.getcwd()

while not cwd.endswith("fast-form"):
    os.chdir("../")
    cwd = os.getcwd()
print(f"CWD is: {cwd}")

In [ ]:
import json
import cv2
import matplotlib.pyplot as plt

In [ ]:
from IPython.display import Image
import PIL.Image

In [ ]:
template_path = "./tests/form_for_test/template.jpg"
document_path = "./tests/form_for_test/document.jpg"

template_path = "./real_forms/justyna_dotazniky/template.pdf"
document_path = "./real_forms/justyna_dotazniky/document.jpg"

template_path = "./real_forms/justyna_dotazniky/PID empty.pdf"
document_path = "./real_forms/justyna_dotazniky/justyna_scan_1.pdf"

In [ ]:
from preprocessing.preprocess import normalize, fit_image_to_templates

from preprocessing.templating import get_templates

In [ ]:
from utils.image_loading import load_images_from_path

In [ ]:
N=0

In [ ]:
%matplotlib inline
# template_img = cv2.imread(template_path)
# plt.imshow(template_img)
# plt.show()
images = load_images_from_path(document_path)
for image in images[:2]:
    plt.figure(figsize = (10,10))
    plt.imshow(image)
    
    plt.show()
    
image = images[N]

In [ ]:
%matplotlib inline
# im = normalize(template_img)
# plt.figure(figsize = (10,10))
# plt.imshow(im)
# plt.show()
normalized_image = normalize(image)
plt.figure(figsize = (10,10))
plt.imshow(normalized_image)
plt.show()

In [ ]:
templates = get_templates(template_path)

In [ ]:
templates_to_use = [templates[N]]

In [ ]:
fitted_image, page = fit_image_to_templates(normalized_image, templates_to_use)
template_image = templates_to_use[0].image

In [ ]:
comparison_image = fitted_image/3 + template_image

In [ ]:
comparison_image = fitted_image/3 + template_image
print(f"Match found for page {page}")
plt.figure(figsize = (10,10))
plt.imshow(comparison_image)
plt.show()

In [ ]:
def add_black_points(img1, img2):
    size1 = img1.shape
    size2 = img2.shape
    
    x_shape = max(size1[0],size2[0])
    y_shape = max(size1[1],size2[1])
    
    empty_new_image = np.zeros((x_shape,y_shape)) + 255
    
    img1_new = empty_new_image.copy()
    img1_new[:size1[0],:size1[1]] = img1
    
    img2_new = empty_new_image.copy()
    img2_new[:size2[0],:size2[1]] = img2
    
    return img1_new, img2_new

In [ ]:
img1, img2 = add_black_points(original_for_show, fitted_image)


print(f"Match found for page {page}")
plt.figure(figsize = (10,10))
plt.imshow(original_for_show)

plt.show()

In [ ]:
import cv2
import numpy as np

def move_image(img,x,y):

    # The number of pixels
    num_rows, num_cols = img.shape[:2]

    # Creating a translation matrix
    translation_matrix = np.float32([ [1,0,x], [0,1,y] ])

    # Image translation
    return cv2.warpAffine(img, translation_matrix, (num_cols,num_rows))

In [ ]:
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

def resize_image(image, ratio):
    return cv2.resize(image, (0,0), fx=ratio, fy=ratio)


In [ ]:
from itertools import product

In [ ]:
angles = np.arange(-5,5,3)
sizes = np.arange(0.9,1.1,0.1)[:1]
movements_x = np.arange(-30,30,20)
movements_y = np.arange(-30,30,20)

In [ ]:
def resize_appropriately(image, template):
    img = cv2.resize(image,(0,0), fx=template.shape[0]/image.shape[0], fy=template.shape[0]/image.shape[0])
    return add_black_points(img, template)

In [ ]:
def convolute(img1, img2):
    return sum(sum((255 - img1)/255 * (255-img2)/255))

In [ ]:
mm = np.float32([[1,0,3],[0,1,5]])

In [ ]:
for i in range(1,200):
    cv2.warpAffine(image, M=mm,dsize = image.shape[1::-1])

In [ ]:
image, template_new = resize_appropriately(normalized_image, template_image)
assert image.shape == template_new.shape
i=0
convol = 0
img_chosen = None
for size in sizes:
    img_resized_only, _ = resize_appropriately(resize_image(image,size), template_new)
    for angle,movement_x,movement_y in product(angles,movements_x, movements_y):
        i+=1
        if i%10 == 0:
            print(angle, movement_x, movement_y)
        img = rotate_image(img_resized_only, angle)
        img = move_image(img,movement_x,movement_y)
        assert img.shape == template_new.shape
        new_convolute = convolute(img, template_new)
        if new_convolute > convol:
            convol = new_convolute
            img_chosen = img
        
        

In [ ]:
comparison_image = img_chosen/3 + template_new
print(f"Match found for page {page}")
plt.figure(figsize = (10,10))
plt.imshow(comparison_image)
plt.show()

In [ ]:
img1, img2 = add_black_points(normalized_image, fitted_image)


print(f"Match found for page {page}")
plt.figure(figsize = (10,10))
plt.imshow(np.clip(510-img1 - img2,a_max=255, a_min=0))

plt.show()

